In [53]:
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
client = OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

In [54]:
#model选用“qwen2.5-coder-3b-instruct”最快，“qwen-coder-plus-lates”其次，"qwen-max/turbo"等极慢
def chatgpt(prompt, model="qwen2.5-coder-3b-instruct")->list:
  messages =[{"role": "user", "content": prompt}]
  outputs = []
  res = client.chat.completions.create(model=model, messages=messages)
  outputs.extend([choice.message for choice in res.choices])
  print('outputs:', outputs)
  return outputs

In [55]:
# 24点游戏：四个数字，每个数字有且仅有一次使用机会，必须使用，利用加减乘除的方法，求出结果为24的组合
propose_prompt = """
输入: 2 8 8 14
可能的下一步行动：
2 + 8 = 10 (left: 8 10 14)
8 / 2 = 4 (left: 4 8 14)
14 + 2 = 16 (left: 8 8 16)
2 * 8 = 16 (left: 8 14 16)
8 - 2 = 6 (left: 6 8 14)
14 - 8 = 6 (left: 2 6 8)
14 /  2 = 7 (left: 7 8 8)
14 - 2 = 12 (left: 8 8 12)
输入: {input}
可能的下一步行动：
"""

value_prompt = """
评估给定的数字是否可以达到24，输出sure(20)/likely(1)/impossible(0.001)，只输出分值
10 14
10 + 14 = 24
left: 20
11 12
11 + 12 = 23
12 - 11 = 1
11 * 12 = 132
11 / 12 = 0.91
left: 0.001
4 4 10
4 + 4 + 10 = 8 + 10 = 18
4 * 10 - 4 = 40 - 4 = 36
(10 - 4) * 4 = 6 * 4 = 24
left: 20
4 9 11
9 + 11 + 4 = 20 + 4 = 24
left: 20
5 7 8
5 + 7 + 8 = 12 + 8 = 20
(8 - 5) * 7 = 3 * 7 = 21
无法达到24, 但数字在合理范围内
left: 1
5 6 6
5 + 6 + 6 = 17
(6 - 5) * 6 = 1 * 6 = 6
无法达到24, 但数字在合理范围内
left: 1
10 10 11
10 + 10 + 11 = 31
(11 - 10) * 10 = 10
10 10 10 太大了
left: 0.001
1 3 3
1 * 3 * 3 = 9
(1 + 3) * 3 = 12
1 3 3 太小了
left: 0.001
{input}
"""

In [56]:
def first_think(input):
  proposals = chatgpt(propose_prompt.format(input=input))[0].content.split('\n')
  proposals = [_ + '\n' for _ in proposals]
  print('first_think_proposals:',proposals)
    # # # 创建对应的index list
  ids = list(range(len(proposals)))
  print('first_think_ids:',ids)
  return ids,proposals


In [57]:
def first_evaluate(proposals):
  values = []
  for proposal in proposals:
    print(proposal)
    current_numbers = proposal.strip().split('\n')[-1].split('left:')[-1].split(")")[0]
    print('first_evaluate_current_numbers:',current_numbers)
    if current_numbers in value_cache:
      values.append(0)
      print('continue loop and first_scores:', values)
      continue
    value_outputs = chatgpt(value_prompt.format(input=current_numbers))
    print(value_outputs,'first_evaluate_value_outputs')
    print('first_evaluate_current_numbers_2：',current_numbers)
    value_cache.append(current_numbers)
    print('value_cache:',value_cache)
    value = 0
    try:
        value = sum([float(_.split('\n')[-1].split('left: ')[-1]) for _ in value_outputs])
    except:
        pass
    values.append(value)
    print('first_values_2:', values)
    # 日志
    temp = proposal.replace("\n", "\\n")
    print(f"[{temp}] 得分: {value}")
  return values

In [61]:
#预测
def first_screen(ids,values):
  # 根据预测结果选择前5个最有可能到达24的候选项
  # 按照评级排序下一步动作的index 并记录对应的下一步动作
  select_ids = sorted(ids, key=lambda i: values[i], reverse=True)[:5]
  select_proposals = [proposals[select_id] for select_id in select_ids]
  print('first_screen_select_proposals:', select_proposals)
  return select_proposals

In [59]:

value_cache= []
input = "5 8 11 13"
print("第一轮：思考生成中......")
ids,proposals = first_think(input)
print("第一轮思考完毕，结果：",ids,proposals)


第一轮：思考生成中......
outputs: [ChatCompletionMessage(content='5 + 8 = 13 (left: 13 11 13)\n8 / 5 = 1.6 (left: 1.6 11 13)\n13 + 5 = 18 (left: 1.6 11 18)\n5 * 8 = 40 (left: 1.6 11 40)\n13 - 5 = 8 (left: 1.6 8 13)\n13 - 5 = 8 (left: 1.6 8 8)\n13 / 5 = 2.6 (left: 1.6 2.6 8)\n13 - 5 = 8 (left: 1.6 8 8)\n13 - 5 = 8 (left: 1.6 8 8)', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None)]
first_think_proposals: ['5 + 8 = 13 (left: 13 11 13)\n', '8 / 5 = 1.6 (left: 1.6 11 13)\n', '13 + 5 = 18 (left: 1.6 11 18)\n', '5 * 8 = 40 (left: 1.6 11 40)\n', '13 - 5 = 8 (left: 1.6 8 13)\n', '13 - 5 = 8 (left: 1.6 8 8)\n', '13 / 5 = 2.6 (left: 1.6 2.6 8)\n', '13 - 5 = 8 (left: 1.6 8 8)\n', '13 - 5 = 8 (left: 1.6 8 8)\n']
first_think_ids: [0, 1, 2, 3, 4, 5, 6, 7, 8]
第一轮思考完毕，结果： [0, 1, 2, 3, 4, 5, 6, 7, 8] ['5 + 8 = 13 (left: 13 11 13)\n', '8 / 5 = 1.6 (left: 1.6 11 13)\n', '13 + 5 = 18 (left: 1.6 11 18)\n', '5 * 8 = 40 (left: 1.6 11 40)\n', '13 - 5 = 8 (left: 1.6 8 13

In [62]:
print("第一轮：评估结果中......")
values = first_evaluate(proposals)
print("第一轮评估完毕，结果：", values)
print("第一轮：筛选候选项......")
select_proposals = first_screen(ids,values)
print("第一轮候选项筛选完毕，结果：", select_proposals)

第一轮：评估结果中......
5 + 8 = 13 (left: 13 11 13)

first_evaluate_current_numbers:  13 11 13
continue loop and first_scores: [0]
8 / 5 = 1.6 (left: 1.6 11 13)

first_evaluate_current_numbers:  1.6 11 13
continue loop and first_scores: [0, 0]
13 + 5 = 18 (left: 1.6 11 18)

first_evaluate_current_numbers:  1.6 11 18
continue loop and first_scores: [0, 0, 0]
5 * 8 = 40 (left: 1.6 11 40)

first_evaluate_current_numbers:  1.6 11 40
continue loop and first_scores: [0, 0, 0, 0]
13 - 5 = 8 (left: 1.6 8 13)

first_evaluate_current_numbers:  1.6 8 13
continue loop and first_scores: [0, 0, 0, 0, 0]
13 - 5 = 8 (left: 1.6 8 8)

first_evaluate_current_numbers:  1.6 8 8
continue loop and first_scores: [0, 0, 0, 0, 0, 0]
13 / 5 = 2.6 (left: 1.6 2.6 8)

first_evaluate_current_numbers:  1.6 2.6 8
continue loop and first_scores: [0, 0, 0, 0, 0, 0, 0]
13 - 5 = 8 (left: 1.6 8 8)

first_evaluate_current_numbers:  1.6 8 8
continue loop and first_scores: [0, 0, 0, 0, 0, 0, 0, 0]
13 - 5 = 8 (left: 1.6 8 8)

first_ev